In [1]:
import pandas as pd
import numpy as np

pm=open('ppm_98_TMRstart.fas','r')
npm=open('npm_98_TMRstart.fas','r')

fas = [[]]

while True:
    reading = pm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_pm=pd.DataFrame(fas,columns=columns)

fas = [[]]

while True:
    reading = npm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_npm=pd.DataFrame(fas,columns=columns)

In [2]:
position = -25

df_pm_freq=df_pm[position].value_counts(sort=True)
df_pm_nor=df_pm[position].value_counts(sort=True,normalize=True)

df_npm_freq=df_npm[position].value_counts(sort=True)
df_npm_nor=df_npm[position].value_counts(sort=True,normalize=True)

printout=pd.concat([df_pm_freq,df_pm_nor,df_npm_freq,df_npm_nor],axis='columns')

printout=printout.set_axis(['pm','pm_normalized','npm','npm_normalized'],axis='columns')

index1=printout.index

printout=printout.assign(AA=index1)

In [3]:
printout

,pm,pm_normalized,npm,npm_normalized,AA
X,32.0,0.410256,395,0.701599,X
T,5.0,0.064103,7,0.012433,T
D,5.0,0.064103,7,0.012433,D
Q,4.0,0.051282,8,0.014210,Q
E,4.0,0.051282,15,0.026643,E
K,4.0,0.051282,8,0.014210,K
P,4.0,0.051282,16,0.028419,P
I,3.0,0.038462,3,0.005329,I
H,2.0,0.025641,4,0.007105,H
A,2.0,0.025641,14,0.024867,A


In [7]:
l_order = {'X': 0, 'L': 1, 'I': 2, 'V': 3,'F': 4, 'P': 5, 'M': 6, 'G': 7, 'A': 8, 'T': 9, 
         'W': 10,'K': 11,'H': 12,'R':13,'D': 14,'E':15, 'Q': 16,'N':17,'C': 18,'S':19,'Y': 20}

printout['order']=printout['AA'].map(l_order)

printout

,pm,pm_normalized,npm,npm_normalized,AA,order
X,32.0,0.410256,395,0.701599,X,0
T,5.0,0.064103,7,0.012433,T,9
D,5.0,0.064103,7,0.012433,D,14
Q,4.0,0.051282,8,0.014210,Q,16
E,4.0,0.051282,15,0.026643,E,15
K,4.0,0.051282,8,0.014210,K,11
P,4.0,0.051282,16,0.028419,P,5
I,3.0,0.038462,3,0.005329,I,2
H,2.0,0.025641,4,0.007105,H,12
A,2.0,0.025641,14,0.024867,A,8


In [9]:
printout=printout.sort_values('order')
printout

,pm,pm_normalized,npm,npm_normalized,AA,order
X,32.0,0.410256,395,0.701599,X,0
L,2.0,0.025641,23,0.040853,L,1
I,3.0,0.038462,3,0.005329,I,2
V,2.0,0.025641,6,0.010657,V,3
F,1.0,0.012821,7,0.012433,F,4
P,4.0,0.051282,16,0.028419,P,5
M,1.0,0.012821,3,0.005329,M,6
G,1.0,0.012821,11,0.019538,G,7
A,2.0,0.025641,14,0.024867,A,8
T,5.0,0.064103,7,0.012433,T,9


In [11]:
#x抜きで集計



X    32.0
L     2.0
I     3.0
V     2.0
F     1.0
P     4.0
M     1.0
G     1.0
A     2.0
T     5.0
K     4.0
H     2.0
R     2.0
D     5.0
E     4.0
Q     4.0
N     1.0
C     NaN
S     2.0
Y     1.0
Name: pm, dtype: float64